Note: \
It might be much faster using asynchronous programming and parsel: import asyncio, import aiohttp, from parsel import Selector, lxml was faster too \
Asyncio with Multiprocessing do this for faster return time for later use \
Even merged parallel (right after a df is returned) instead of merging one of byafter we get all the dataframe at the end \

checkout for race condition (is there any race conditon? the shared data might be updated at the same time, but no share of the data here so good) 


In [11]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import lxml

Note: you may need to restart the kernel to use updated packages.


In [1]:
# class getLeagueTable(object):
    
#     def __init__(self, leagueName):
#         self.leagueName = leagueName
#         self.url = ""
        
#     def getUrl(self):
#         if self.leagueName.lower() == "serie a":
#             self.url = "https://fbref.com/en/comps/11/Serie-A-Stats"
#         elif self.leagueName.lower() == "premier league":
#             self.url = "https://fbref.com/en/comps/9/Premier-League-Stats"
#         elif self.leagueName.lower() == "laliga":
#             self.url = "https://fbref.com/en/comps/12/La-Liga-Stats"
#         elif self.leagueName.lower() == "bundesliga":
#             self.url = "https://fbref.com/en/comps/20/Bundesliga-Stats"
#         elif self.leagueName.lower() == "league 1":
#             self.url = "https://fbref.com/en/comps/13/League-1-Stats"
#         return self.url
    
#     def getTable(self):
#         a = self.getUrl()
#         club = requests.get(self.getUrl())
#         # get Scores (on the same page)
#         scores = pd.read_html(club.text)
#         scores = pd.read_html(club.text, match = "Regular season Table")[0]
#         keep = ["Rk", "Squad", "MP", "W", "D", "L", "GF", "GD", "Pts", "Top Team Scorer"]        
#         scores = scores[keep]
#         names = {"Rk": "Rank", "Squad": "Team", "Pts":"Points"}
#         scores.rename(columns=names, inplace=True)  # Rename columns
#         return scores

# tb = getLeagueTable("Bundesliga")
# tb.getTable().to_html()

In [19]:
# Create your views here.
# from .models import myModel
import csv

import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import concurrent.futures
import time
import threading

class LeagueTable(object):

    def __init__(self):
        self.url = "https://fbref.com/en/comps/9/Premier-League-Stats"
        # table = pd.DataFrame()

    def getUrl(self):
        return self.url
    # def getTable(self):
    #     return self.table
    
    # def setTable(self, df):
    #     self.table = df

    def getAllClubLinks(self, url):
        #get the response
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        #select the first table and find all anchor tags
        teamTable = soup.select("table.stats_table")[0]
        teamLinks = teamTable.find_all("a")
        
        #get if link has href and and "/squads/"
        teamLinks = [l.get("href") for l in teamLinks]
        teamLinks = [l for l in teamLinks if "/squads/" in l]
        teamLinks = [f"https://fbref.com{l}" for l in teamLinks]
        print(teamLinks)
        return teamLinks

    def getPLData(self, url):
        lock = threading.Lock()
        # runs only once for all clubs, returns team links
        allClubLinks = self.getAllClubLinks(url)
        names = {"Comp": "Competition", "Poss": "Possession", "Sh":"Shots", "SoT":"Shots Target", "CrdY":"Yellow", "CrdR":"Red", "Fls":"Fouls","Off":"Offside"}
        
        result = pd.DataFrame()  # Initialize an empty dataframe
        
        with concurrent.futures.ThreadPoolExecutor() as executor:
            #submit each club processing as a separate task
            club_tasks = [executor.submit(self.getClubData, link) for link in allClubLinks]
            
            # Process the completed tasks as they finish
            for future in concurrent.futures.as_completed(club_tasks):
                club_data = future.result()
                if club_data is not None:
                    with lock:
                        result = pd.concat([result, club_data], axis=0, ignore_index=True)  # Concatenate current club_data with the result dataframe
#                         print(result)

        result.rename(columns=names, inplace=True)  # Rename columns
        return result

    def getAdditionalData(self, requestData, param, match, keepList):
        soup = BeautifulSoup(requestData.text, 'html.parser')
        all_links = soup.find_all("a")
        links = [l.get("href") for l in all_links]
        links = [l for l in links if l and param in l]
        links = "https://fbref.com" + links[0]
        time.sleep(random.randint(10,35))
        shooting = requests.get(links)
        shooting = pd.read_html(shooting.text, match=match)[0]
        shooting.columns = shooting.columns.droplevel()
        shooting = shooting[keepList]
        return shooting[:-1]

    def getClubData(self,link):
        time.sleep(random.randint(10,35))
        club = requests.get(link)
        clubName = link.split("/")[-1].replace("-Stats","").replace("-", " ")
        print(f'Getting Data for {clubName}')

        # get Scores (on the same page)
        scores = pd.read_html(club.text)
        scores = pd.read_html(club.text, match = "Scores & Fixtures")[0]
        scores = scores[['Date', 'Comp', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent', 'Poss']]

        # get shooting and other stat (figure better ways for these two)
        shooting = self.getAdditionalData(club, "/all_comps/shooting/", "Shooting", ['Date', 'Sh', "SoT"])
        time.sleep(random.randint(10,35))
        misc = self.getAdditionalData(club, "/all_comps/misc/", "Miscellaneous", ['Date', 'CrdY', 'CrdR', 'Fls', 'Off'])
        time.sleep(random.randint(10,35))

        finalClubData = scores.merge(shooting, how='left').merge(misc, how='left')
        finalClubData = finalClubData[finalClubData["Comp"] == "Premier League"]
        finalClubData["Team"] = clubName
        finalClubData['Date'] = pd.to_datetime(finalClubData['Date'])
        finalClubData['Date'] = finalClubData['Date'].dt.strftime('%d-%b-%Y')
        print(f'Data for {clubName} gathered')
        return finalClubData

#download the data
#create table object
table = LeagueTable()
res = table.getPLData(table.getUrl())
res.to_csv("PLData.csv", index = False)

['https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats', 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats', 'https://fbref.com/en/squads/19538871/Manchester-United-Stats', 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats', 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats', 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats', 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats', 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats', 'https://fbref.com/en/squads/cd051869/Brentford-Stats', 'https://fbref.com/en/squads/fd962109/Fulham-Stats', 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats', 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats', 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats', 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats', 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats', 'https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats', 'https://fbref.com/en/squads/

KeyboardInterrupt: 